In [8]:
import pandas as pd

In [29]:
# kospi(유가증권)
#kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', index_col=["종목코드"], header=0)[0]
kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', header=0)[0]
print(kospi_df.shape)
# kosdaq
kosdaq_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=kosdaqMkt', header=0)[0]
print(kosdaq_df.shape)
# konexMkt
konex_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=konexMkt', header=0)[0]
print(konex_df.shape)

(777, 9)
(1268, 9)
(148, 9)


In [30]:
kospi_df['marketType'] = 'kospi'
kosdaq_df['marketType'] = 'kosdaq'
konex_df['marketType'] = 'konex'

In [31]:
frames = [kospi_df, kosdaq_df, konex_df]
stock_df = pd.concat(frames, ignore_index=True)
print(stock_df.shape)

(2193, 10)


In [32]:
# 6자리 고정으로 left zero padding하여 종목코드 정제하기(ex) 5404 -> 005404)
stock_df['종목코드'] = stock_df['종목코드'].apply(lambda x: str(x).zfill(6))

In [33]:
feature_names = ['회사명', '종목코드', '업종', '결산월', 'marketType']
stock_train = stock_df[feature_names]

print(stock_train.shape)
stock_train.tail(5)

(2193, 5)


,회사명,종목코드,업종,결산월,marketType
2188,유디피,091270,소프트웨어 개발 및 공급업,12월,konex
2189,젠큐릭스,229000,의료용 기기 제조업,12월,konex
2190,청광종건,140290,토목 건설업,12월,konex
2191,탈모닷컴,284610,기타 화학제품 제조업,12월,konex
2192,피엔아이시스템,242350,"영화, 비디오물, 방송프로그램 제작 및 배급업",12월,konex


# 모든 종목 코드 갖고 오기

In [34]:
def get_all_codes():
    # kospi(유가증권)
    #kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', index_col=["종목코드"], header=0)[0]
    kospi_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=stockMkt', header=0)[0]
    # kosdaq
    kosdaq_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=kosdaqMkt', header=0)[0]
    # konexMkt
    konex_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=konexMkt', header=0)[0]
    
    kospi_df['marketType'] = 'kospi'
    kosdaq_df['marketType'] = 'kosdaq'
    konex_df['marketType'] = 'konexMkt'
    print(kospi_df.shape, kosdaq_df.shape, konex_df.shape)
    
    frames = [kospi_df, kosdaq_df, konex_df]
    stock_df = pd.concat(frames, ignore_index=True)

    # 종목코드 6자리 맞추기
    stock_df['종목코드'] = stock_df['종목코드'].apply(lambda x: str(x).zfill(6))
    return stock_df

In [35]:
codes = get_all_codes()

(777, 10) (1268, 10) (148, 10)


In [36]:
print(codes.shape)
codes.head()

(2193, 10)


,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,marketType
0,BYC,001460,봉제의복 제조업,"메리야스,란제리 제조,도매/건축공사/부동산 임대,분양,공급",1975-06-02,12월,유 중 화,http://www.byc.co.kr,서울특별시,kospi
1,CJ CGV,079160,"영화, 비디오물, 방송프로그램 제작 및 배급업","영화상영,영화관 운영",2004-12-24,12월,서 정,http://www.cgv.co.kr,서울특별시,kospi
2,DB손해보험,005830,보험업,"자동차보험,화재보험,해상보험,특종보험,장기보험",1973-06-28,12월,김정남,http://www.idbins.com,서울특별시,kospi
3,DSR제강,069730,1차 철강 제조업,"와이어로프,각종 경강선,철선제품,PC강선,아연도 강연선 제조",2003-01-28,12월,홍하종,http://www.dsrcorp.com,전라남도,kospi
4,KB금융,105560,기타 금융업,-,2008-10-10,12월,윤종규,http://www.kbfg.com,서울특별시,kospi


## 특정 회사 재무제표 갖고 오기

In [37]:
# 재무제표 년/월 얻어오기
import re
from datetime import datetime

def get_date_str(s):
    date_str = ''
    r = re.search("\d{4}/\d{2}", s)
#     r = re.search("\d{4}", s)
    if r:
        date_str = r.group()

    return date_str

In [38]:
def get_financial_statement(code, ifrs_type=4, unit='Y'):
    url_tmpl = 'http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=%s&fin_typ=%s&freq_typ=%s'
    url = url_tmpl % (code, ifrs_type, unit) # 삼성전자, 4(IFRS 연결), Y:년 단위 / Q: 분기
    print(url)

    dfs = pd.read_html(url, encoding='utf8')
    df = dfs[0]
    
    # multi columns 제거
    cols = list(df.columns.droplevel(0))
    cols = [get_date_str(col) for col in cols]
    
    df = df.set_index('주요재무정보')
    
    # 컬럼이름 변경
    df.columns = cols[0:-1]
    
    # index 변환
    indexes = []
    for i in range(0, df.index.size):
        indexes.append(df.index[i][0])
        
    new_index_dict = dict(zip(df.index, indexes))
    df.rename(new_index_dict, axis='index', inplace=True)
    
    # NaN을 0.0 숫자로 바꾸기
    df.fillna(0, inplace=True)
    
    return df

In [39]:
print(codes.loc[0]['종목코드'], codes.loc[0]['회사명'])

001460 BYC


In [40]:
df = get_financial_statement(codes.loc[0]['종목코드'])
df.head()

http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=001460&fin_typ=4&freq_typ=Y


,2013/12,2014/12,2015/12,2016/12,2017/12,2018/12,2019/12,2020/12
주요재무정보,,,,,,,,
매출액,1824.0,1769.0,1822.0,2119.0,1958.0,0.0,0.0,0.0
영업이익,251.0,279.0,211.0,162.0,172.0,0.0,0.0,0.0
영업이익(발표기준),251.0,279.0,211.0,162.0,172.0,0.0,0.0,0.0
세전계속사업이익,219.0,249.0,177.0,207.0,96.0,0.0,0.0,0.0
당기순이익,155.0,190.0,132.0,175.0,62.0,0.0,0.0,0.0


## 엑셀에 저장하기

In [23]:
# 항목별로 다른 sheet에 저장하기
def save_fincial_statement(file_name, sheet_names, inquiry_years, code, corp_nm):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
        for idx, val in enumerate(sheet_names):
            data = df[idx:idx+1][inquiry_years]
            data.insert(0, 'corp_nm', corp_nm)
            data.index.rename("", inplace=True)
            data.index = [code]
#             print(data)
            data.to_excel(writer, index=True, sheet_name=sheet_names[idx])
        writer.save()

## 엑셀에 데이터 추가하기

In [25]:
from openpyxl import load_workbook

In [26]:
def append_financial_statement(df, file_name, code, corp_nm):
    book = load_workbook(file_name)
    writer = pd.ExcelWriter(file_name, engine='openpyxl')
    writer.book = book
    for sheet_name in df.index.values:
#         print(sheet_name, df.loc[sheet_name]['2016/12'], df.loc[sheet_name]['2017/12'])
        writer.book[sheet_name].append([code, corp_nm, df.loc[sheet_name]['2016'], df.loc[sheet_name]['2017']])
    writer.save()

## 모든 회사의 재무제표 읽기

In [27]:
def get_all_finacial_statements(file_name, datas):    
    for idx, row in datas.iterrows():
        code = datas.loc[idx]['종목코드']
        corp_nm = datas.loc[idx]['회사명']
        df = get_financial_statement(code, 4, 'Y')
        if idx == 0:
            save_fincial_statement(file_name, df.index.values, ['2016', '2017'], code, corp_nm)
        else:
            append_financial_statement(df, file_name, code, corp_nm)
        if idx % 500 == 0:
            print(df.head(3))

In [28]:
get_all_finacial_statements('test.xlsx', codes)

http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=068400&fin_typ=4&freq_typ=Y
              2013    2014    2015    2016    2017  2018  2019  2020
주요재무정보                                                              
매출액         4757.0  5700.0  6321.0  6476.0  6368.0   0.0   0.0   0.0
영업이익         445.0   437.0   403.0   327.0   409.0   0.0   0.0   0.0
영업이익(발표기준)   445.0   437.0   403.0   327.0   409.0   0.0   0.0   0.0
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=139130&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=017940&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=114090&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=083420&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_cd=000040&fin_typ=4&freq_typ=Y
http://companyinfo.stock.naver.com/v1/company/ajax/cF1001.aspx?cmp_

ValueError: Cannot convert 2017    716.0
2017    629.0
Name: 매출액, dtype: float64 to Excel

In [54]:
codes[0:2]

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역,marketType
0,AJ렌터카,068400,운송장비 임대업,승용자동차 임대,2012-07-27,12월,홍성관,http://www.ajrentacar.co.kr,서울특별시,kospi
1,DGB금융지주,139130,기타 금융업,지주회사,2011-06-07,12월,박인규,http://www.dgbfg.co.kr,대구광역시,kospi
